metric: The number of pages visited during their first session since the test started. number increased, test success
<br>
1. Is the test winning? That is, should 100% of the users see the Recommended Friends feature?
2. Is the test performing similarly for all user segments or are there differences among different segments?
3. If you identified segments that responded differently to the test, can you guess the reason? Would this change your point 1 conclusions?

## Load Data and Library

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve,precision_recall_curve,auc
from sklearn.cluster import KMeans
from sklearn import preprocessing
import pandasql as ps
import h2o
from h2o.frame import H2OFrame
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.grid.grid_search import H2OGridSearch
from sklearn.ensemble import IsolationForest
from scipy.stats import ttest_ind

%matplotlib inline

## Data Exploratory Analysis

In [3]:
test = pd.read_csv("test_table.csv")
user = pd.read_csv("user_table.csv")
df = test.merge(user, on='user_id')

In [4]:
df.head()

,user_id,date,browser,test,pages_visited,signup_date
0,600597,2015-08-13,IE,0,2,2015-01-19
1,4410028,2015-08-26,Chrome,1,5,2015-05-11
2,6004777,2015-08-17,Chrome,0,8,2015-06-26
3,5990330,2015-08-27,Safari,0,8,2015-06-25
4,3622310,2015-08-07,Firefox,0,1,2015-04-17


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 6 columns):
user_id          100000 non-null int64
date             100000 non-null object
browser          100000 non-null object
test             100000 non-null int64
pages_visited    100000 non-null int64
signup_date      100000 non-null object
dtypes: int64(3), object(3)
memory usage: 5.3+ MB


In [9]:
df.nunique()

user_id          100000
date                 31
browser               5
test                  2
pages_visited        18
signup_date         243
dtype: int64

In [5]:
df.isnull().sum() #no missing values

user_id          0
date             0
browser          0
test             0
pages_visited    0
signup_date      0
dtype: int64

In [23]:
df.test.value_counts()

1    50154
0    49846
Name: test, dtype: int64

In [6]:
#Is the test winning? That is, should 100% of the users see the Recommended Friends feature?
df.groupby('test')['pages_visited'].mean().reset_index()
#test not wining

,test,pages_visited
0,0,4.608394
1,1,4.599693


Is the test performing similarly for all user segments or are there differences among different segments?

In [16]:
df.groupby(['test','browser'])['pages_visited'].mean().reset_index()
#opera

,test,browser,pages_visited
0,0,Chrome,4.613341
1,0,Firefox,4.600164
2,0,IE,4.598478
3,0,Opera,4.546438
4,0,Safari,4.638180
5,1,Chrome,4.690680
6,1,Firefox,4.714259
7,1,IE,4.685985
8,1,Opera,0.000000
9,1,Safari,4.692336


In [18]:
df['signup_date'] = pd.to_datetime(df['signup_date'])

In [19]:
df['signup_year']=df['signup_date'].dt.strftime('%Y')
df['signup_month']=df['signup_date'].dt.strftime('%m')

In [22]:
df.groupby(['test','signup_month'])['pages_visited'].mean().reset_index()

,test,signup_month,pages_visited
0,0,01,4.656550
1,0,02,4.574011
2,0,03,4.594084
3,0,04,4.631448
4,0,05,4.571031
5,0,06,4.616448
6,0,07,4.576759
7,0,08,4.628867
8,1,01,4.616401
9,1,02,4.621673


## Model

In [27]:
# perform student's t-test
control_page = df[df['test'] == 0]['pages_visited'].values
experiment_page = df[df['test'] == 1]['pages_visited'].values

print('T-test:\t', ttest_ind(a=control_page, b=experiment_page, equal_var=False))

T-test:	 Ttest_indResult(statistic=0.5571118435554797, pvalue=0.5774523171559118)
